# IML Project Phase 2


In [20]:
import numpy as np
import cupy as cp
import matplotlib.pyplot as plt
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from torchvision import transforms
from scipy.stats import multivariate_normal
import os
import cv2 as cv
import random

In [21]:
def extract_patch(this=None, path='Dataset', m=8, corrupted=False, idx=None):

    subfolder = 'corrupted' if corrupted else 'original'
    files = os.listdir(f'./{path}/MNIST-m={m}/MNIST/{subfolder}')
    patches = []

    if this == None:
        
        for i, file in enumerate(files):
            if idx is None or i in idx:
                img = cv.imread(f'./{path}/MNIST-m={m}/MNIST/{subfolder}/{file}')
                img = cv.cvtColor(img, cv.COLOR_RGB2GRAY)
                x, y = img.shape

                for i in range(x - m + 1):
                    for j in range(y - m + 1):
                        patches.append(img[i : i + m, j : j + m].reshape((m**2, 1)))
    else:
        for i, img in enumerate(this):
            if idx is None or i in idx:
                x, y = img.shape
                for i in range(x - m + 1):
                    for j in range(y - m + 1):
                        patches.append(img[i : i + m, j : j + m].reshape((m**2, 1)))
    
    return(patches)



In [22]:
def E_step(data, mu, sigma, pi, n):

    Q = np.asarray([pi[i] * multivariate_normal.pdf(data, mu[i,:], sigma[i,:,:]) for i in range(n)]).T
    Q = Q / np.reshape(np.sum(Q, axis=1),(len(data), 1))

    return Q

def M_step(data, sigma, Q, n):

    m = data.shape[1]
    mu =  np.asarray([np.average(data, weights=Q[:,i], axis=0) for i in range(n)])

    for i in range(n):
        
        sigma[i, :, :] = (data - mu[i,:]).T @ np.multiply((data - mu[i,:]), np.tile(Q[:,i], (m, 1)).T)
        sigma[i, :, :] = (sigma[i, :, :] + sigma[i, :, :].T)  / np.sum(Q[:,i]) / 2.0
        L, D = np.linalg.eig(sigma[i, :, :])
        D = np.real(D)
        L = np.real(L)
        sigma[i, :, :] = D @ np.diag(np.maximum(0.001, L)) @ D.T + np.eye(sigma[i, :, :].shape[0])/1000.0
            
    pi = np.mean(Q, axis=0)
    return mu, sigma, pi


def cluster_match_percent(Q):
    calc_clusters = np.argmax(Q, axis=1)
    indexes = [np.median(calc_clusters[:200]),np.median(calc_clusters[200:400]),np.median(calc_clusters[400:])]
    real_clusters = np.repeat(indexes,200,axis=0)
    return (1 - np.sum((real_clusters - calc_clusters) != 0) / 600) * 100




In [23]:
train_set = MNIST(root='.', train=True, download=True, transform=transforms.ToTensor())
train_loader = DataLoader(train_set, 64, shuffle=True)


In [24]:
mySet = []
i = 0
for x, _ in train_loader:
    if random.choice([True, False]):
        i += 1
        x, label = train_set[i]
        x = x.numpy().squeeze()
        mySet.append(x)
        if i == 1000:
            break

In [27]:

m = 4
data = extract_patch(this=mySet, m=m,corrupted=True)
data = np.asarray(data).reshape(len(data), len(data[0]))
x, y = mySet[0].shape
n = 10                       

mu = data[np.random.randint(100, data.shape[0], n), :] 
print(mu)
sigma = np.tile(np.cov(data.T), (n,1,1))
pi = np.repeat(1/n, n, axis = 0)

dif = 1000
step = 1

while dif > 0.01:

    Q = E_step(data, mu, sigma, pi, n)
    new, sigma, pi = M_step(data, sigma, Q, n)
    dif = np.linalg.norm(new - mu)
    mu = new
    print(f'step {step}\t\t Mu diff: {round(dif,4)}')
    step += 1


[[0.78431374 0.9764706  0.9882353  0.36078432 0.9882353  0.9882353
  0.5568628  0.         0.9882353  0.78431374 0.04313726 0.
  0.         0.         0.         0.        ]
 [0.99215686 0.99215686 0.9607843  0.7294118  0.99215686 0.91764706
  0.30980393 0.05490196 0.9607843  0.30980393 0.         0.
  0.5058824  0.         0.         0.21176471]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.23529412 0.9843137  0.5294118  0.01176471 0.25882354 0.99607843
  0.3137255  0.         0.25882354 0.99607843 0.3137255  0.
  0.25882354 0.9960784

In [28]:
print(mu)

[[7.49423066e-01 7.26574615e-01 5.08588178e-01 3.08004421e-01
  9.72254705e-01 8.48739403e-01 4.75971274e-01 2.51597757e-01
  9.81831749e-01 7.94358791e-01 3.78885181e-01 2.11609756e-01
  7.61137800e-01 5.18594755e-01 2.60387751e-01 1.85123337e-01]
 [5.87608182e-01 4.19031262e-01 3.16204565e-01 2.87158103e-01
  3.91939026e-01 1.79007567e-01 1.02311787e-01 9.67017191e-02
  2.06949506e-01 1.52254627e-02 7.61704683e-04 1.82077900e-03
  1.38274187e-01 5.28074931e-03 2.11078038e-04 1.63205771e-03]
 [2.40009762e-03 1.93113867e-04 1.68011129e-04 1.27213970e-03
  4.07403165e-04 3.87226086e-06 2.90512139e-06 3.01003585e-04
  3.48695301e-04 7.26905010e-06 4.12403612e-06 3.54574320e-04
  1.34364883e-03 1.81926389e-04 2.14968255e-04 2.60112684e-03]
 [2.40009762e-03 1.93113867e-04 1.68011129e-04 1.27213970e-03
  4.07403165e-04 3.87226086e-06 2.90512139e-06 3.01003585e-04
  3.48695301e-04 7.26905010e-06 4.12403612e-06 3.54574320e-04
  1.34364883e-03 1.81926389e-04 2.14968255e-04 2.60112684e-03]
 [2.

In [31]:
def Z_Y(Y, mu, sigma, W, s):
    sigmaY = np.diag(s**2) + W @ sigma @ W.T
    sigmaZY = np.linalg.inv(sigma.I + W.T @ sigmaY.I @ W)
    muZY = sigmaZY @ (W.T @ sigmaY.I @ Y + sigma.I @ mu)

W = np.load(f'./Dataset/MNIST-m={m}/MNIST/W.npy')

Y = extract_patch(m = m, corrupted=True, idx=[0])
l = len(Y)
Y = np.asarray(Y, dtype=float).reshape(l, len(Y[0])) / 255

K = np.asarray([pi[i] * multivariate_normal.pdf(Y, mu[i,:], sigma[i,:,:]) for i in range(n)]).T

K = np.argmax(K, axis=1)
newY = []
for i in range(l):
    newY.append(mu[K[i], :])



SyntaxError: expected ':' (1473559713.py, line 16)